In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from matplotlib import pyplot as plt
%matplotlib inline

In [94]:
def rank(df:pd.DataFrame, data_field="data", rank_field="rank"):
    rank = []
    uvalues = np.sort(df[data_field].unique())
    for el in uvalues:
        indexes = np.where(df[data_field] == el)[0] + 1
        m_rank = np.mean(indexes)
        rank.extend([m_rank]*len(indexes))
    df[rank_field] = rank
    return df

def calculate_mode(alpha, T, mode):
    if mode == "ne":
        t_alpha_l = norm.ppf(alpha/2)
        t_alpha_r = norm.ppf(1 - alpha/2)
        p_value = min(2*norm.cdf(T),2 - 2*norm.cdf(T))
        
        if T > t_alpha_l and T < t_alpha_r:
            # лежит внутри диаазона
            return True, p_value, T
    elif mode == "gt":
        t_alpha = norm.ppf(1 - alpha)
        p_value = 1 - norm.cdf(T)
        if T < t_alpha:
            return True, p_value, T
        else:
            return False, p_value, T
        
    elif mode == "lt":
        t_alpha = norm.ppf(alpha)
        p_value = norm.cdf(T)
        if T > t_alpha:
            return True, p_value, T
        else:
            return False, p_value, T
    else:
        print('Wrong mode!')
        return None
    
def check_hypothesis(data, mode, alpha=0.05):
    """
    Проверка гипотезы
    
    data - словарь с результатами двух экспериментов
    alpha - уровень значимости
    mode - тип гипотезы Н1:
        "ne" - not equal - гипотеза о неравенстве
        "gt" - greater than
        "lt" - less than
        
    Возвращает
        True/False - принимаем гипотезу или нет
        p_value
        T
    """
    n1_l = data['n11'] + data['n21']
    n1_r = data['n11'] + data['n12']
    n2_l = data['n12'] + data['n22']
    n2_r = data['n21'] + data['n22']
    n = n1_r + n2_r
    
    T = (data["n11"]/n1_r - data["n12"]/n2_r) / np.sqrt( n1_l/n * (1 - n1_l/n) * (1/n1_r + 1/n2_r))
    
    return calculate_mode(alpha, T, mode)
    
def check_hypothesis_means(data1: pd.DataFrame, data2:pd.DataFrame, mode, alpha=0.05):
    """
    Проверка гипотезы
    
    data1 - массив с результатами 1го эксперимента
    data2 - массив с результатами 2го эксперимента
    alpha - уровень значимости
    mode - тип гипотезы Н1:
        "ne" - not equal - гипотеза о неравенстве
        "gt" - greater than
        "lt" - less than
        
    Возвращает
        True/False - принимаем гипотезу или нет
        p_value
        T
    """
    data1["label"] = "1"
    data2["label"] = "2"
    m = data1.shape[0]
    n = data2.shape[0]
    total_df = data1.append(data2)
    total_df.sort_values(by="data", inplace=True)
    total_df.reset_index(inplace=True, drop=True)
    total_df = rank(total_df)
    
    W = sum(total_df[total_df["label"] == "2"]["rank"])
    M = n/2 * (m + n + 1)
    D = n*m/12 * (m + n + 1)
    W = (W - M) / np.sqrt(D)
    return calculate_mode(alpha, W, mode)
        

In [57]:
# Задача 1
data = {}
data["n11"] = 42
data["n12"] = 105 - 42
data["n21"] = 65
data["n22"] = 195 - 65

result = check_hypothesis(data, mode = "ne")
print("Принимаем гипотезу:", result[0])
print(f"p-value: {result[1]:.3f}")
print(f"статистика: {result[2]:.3f}")

Принимаем гипотезу: True
p-value: 0.185
статистика: 1.327


In [58]:
# Задача 2
data = {}
data["n11"] = 172
data["n12"] = 3
data["n21"] = 168
data["n22"] = 32

result = check_hypothesis(data, mode = "gt")
print("Принимаем гипотезу:", result[0])
print(f"p-value: {result[1]:.3f}")
print(f"статистика: {result[2]:.3f}")

Принимаем гипотезу: False
p-value: 0.000
статистика: 32.143


In [95]:
# Задача 3
data1 = pd.DataFrame({"data":[130, 110, 120, 140, 200, 130, 140, 170, 160, 140]})
data2 = pd.DataFrame({"data":[120, 190, 130, 160, 150, 120, 110, 120, 200]})

result = check_hypothesis_means(data1,data2,mode = "gt")
print("Принимаем гипотезу:", result[0])
print(f"p-value: {result[1]:.3f}")
print(f"статистика: {result[2]:.3f}")

Принимаем гипотезу: True
p-value: 0.628
статистика: -0.327


In [96]:
# Задача 4
data1 = pd.DataFrame({"data":[102.4, 100.0, 67.6, 65.9, 64.7, 39.6, 31.2]})
data2 = pd.DataFrame({"data":[48.1, 45.5, 41.7, 35.4, 29.1, 18.9, 58.3, 68.8, 71.3, 94.3]})

result = check_hypothesis_means(data1,data2,mode = "ne")
print("Принимаем гипотезу:", result[0])
print(f"p-value: {result[1]:.3f}")
print(f"статистика: {result[2]:.3f}")

Принимаем гипотезу: True
p-value: 0.283
статистика: -1.073
